In [ ]:
import os
import requests
from bs4 import BeautifulSoup

url_base = "https://"Insira-o-link-do-site-aqui"/"

# Nome do arquivo de entrada (consultas) e arquivo de saída (resultados)
input_filename = "DscProduto.txt"
output_filename = "resultados.txt"
processed_queries_filename = "consultas_ja_realizadas.txt"  # Arquivo para consultas já processadas
error_filename = "consultas-erro-codificacao.txt"  # Arquivo para consultas com erro de codificação

# Verifique se o arquivo de entrada existe
if not os.path.isfile(input_filename):
    print(f"Arquivo de entrada '{input_filename}' não encontrado.")
    exit(1)

# Crie um conjunto para manter o controle das consultas já realizadas
consultas_ja_realizadas = set()

# Crie os arquivos de consultas já realizadas e consultas com erro de codificação se eles não existirem
if not os.path.isfile(processed_queries_filename):
    open(processed_queries_filename, "w").close()

if not os.path.isfile(error_filename):
    open(error_filename, "w").close()

# Carregue as consultas já realizadas a partir do arquivo, se existir
if os.path.isfile(processed_queries_filename):
    with open(processed_queries_filename, "r") as processed_queries_file:
        consultas_ja_realizadas = set(line.strip() for line in processed_queries_file)

# Abra o arquivo de resultados fora do loop para evitar a sobrescrição
with open(output_filename, "a") as output_file:
    # Abra o arquivo de consultas para leitura
    with open(input_filename, "r") as input_file:
        # Coletar resultados dos títulos e preços limitando resultados especificamente por consulta
        for query in input_file:
            query = query.strip()  # remove espaços em branco e quebras de linha

            # Verifique se a consulta já foi processada
            if not query:
                continue  # Pule consultas em branco

            # Verifique se a consulta já foi processada
            if query in consultas_ja_realizadas:
                continue  # Pule consultas já processadas

            try:
                soup = BeautifulSoup(requests.get(url_base + query).content, "html.parser")

                count = 0  # Variável de contagem para limitar os resultados

                while True:
                    for tag_title, tag_price in zip(soup.select(".ui-search-item__title"), soup.select("span.andes-money-amount")):
                        count += 1
                        result = f"Item {count}\nTitulo: {tag_title.text}\nPreço: {tag_price.text}"
                        output_file.write(result + "\n\n")  # Adicione o resultado ao arquivo de resultados

                        if count >= 3:   # Encerra o loop após atingir o número de resultados especificados
                            break

                    if count >= 3:
                        break

                    next_link = soup.select_one("a.andes-pagination__link:-soup-contains(Seguinte)")
                    if not next_link:
                        break

                    soup = BeautifulSoup(requests.get(next_link["href"]).content, "html.parser")

                # Registre a consulta como processada no conjunto de consultas já realizadas
                consultas_ja_realizadas.add(query)

                # Escreva imediatamente a consulta no arquivo de consultas já realizadas
                with open(processed_queries_filename, "a") as processed_queries_file:
                    processed_queries_file.write(query + "\n")

            except UnicodeDecodeError:
                print(f"Erro de decodificação para a consulta: {query.strip()}")
                with open(error_filename, "a") as error_file:
                    error_file.write(query + "\n")  # Salve a consulta com erro de codificação no arquivo de erros
                continue  # Pule esta consulta e continue com a próxima

            except requests.RequestException as e:
                print(f"Erro na consulta: {query.strip()}")
                print(f"Detalhes do erro: {str(e)}")

print(f"Resultados adicionados a '{output_filename}'.")


Resultados adicionados a 'resultados.txt'.
